# Clasificar personas

In [118]:
import numpy as np
import pandas as pd
import os
from tqdm.auto import tqdm

In [119]:
directory = '../Deteccion/RetinaFace/Friends'

In [164]:
class Personas:
    
    existentes = []
    path = ''
    thr = 0.3
    directory = '../Deteccion/RetinaFace/Friends'
    EMOCIONES = [
        "angry", 
        "disgust", 
        "scared", 
        "happy", 
        "sad", 
        "surprised", 
        "neutral"
    ]
    
    def __init__(self, descriptor, frame, index, known_identity=False):
        self.descriptores = []
        self.data = pd.DataFrame(columns=[
            "Frame",
            "Index",
            "angry", 
            "disgust", 
            "scared", 
            "happy", 
            "sad", 
            "surprised", 
            "neutral",
            "max_emotion",
            "yaw",
            "pitch",
            "roll",
            "Mira A (Index)",
            "bb_x_min", 
            "bb_y_min", 
            "bb_x_max", 
            "bb_y_max"
        ])
        if known_identity:
            for desc in descriptor:
                self.add_descriptor(desc)
        else:
            self.add(descriptor, frame, index)
        
    def add_descriptor(self, descriptor):
        normalizado = self.normalizar(descriptor)
        self.descriptores.append(normalizado)
    
    def add_emocion(self, frame, index):
        emociones = np.load(f'{directory}/{frame}/emociones.npy')
        self.data.loc[frame, self.EMOCIONES] = emociones[index]
        
        max_emotion = np.argmax(emociones[index])
        self.data.loc[frame, 'max_emotion'] = self.EMOCIONES[max_emotion]
    
    def add_headpose(self, frame, index):
        headpose = np.load(f'{directory}/{frame}/headpose.npy')
        self.data.loc[frame, ['yaw', 'pitch', 'roll']] = headpose[index]
    
    def add_bb(self, frame, index):
        bb = np.load(f'{directory}/{frame}/bounding_boxes.npy')
        self.data.loc[frame, ['bb_x_min', 'bb_y_min', 'bb_x_max', 'bb_y_max']] = bb[index].astype(int)
    
    def add_interaccion(self, frame, index):
        interacciones = np.load(f'{directory}/{frame}/interacciones_headpose.npy')
        interacciones_origen = [tup[0] for tup in interacciones]
        interacciones_destino = [tup[1] for tup in interacciones]
        
        # Revisar si persona se encuentra mirando a alguien
        try:
            ix_person = interacciones_origen.index(index)
            
            # Si encuentra a alguien entonces asignar destino
            destino = interacciones_destino[ix_person]
            
        except ValueError:
            destino = None
        
        self.data.loc[frame, 'Mira A (Index)'] = destino
            
    def add(self, descriptor, frame, index):
        self.data.loc[frame, "Frame"] = frame
        self.data.loc[frame, "Index"] = index
        self.add_descriptor(descriptor)
        self.add_emocion(frame, index)
        self.add_headpose(frame, index)
        self.add_bb(frame, index)
        self.add_interaccion(frame, index)
    
    @staticmethod
    def normalizar(descriptor):
        if len(descriptor.shape) > 1:
            return (descriptor.T / np.linalg.norm(descriptor, axis= 1)).T
        else:
            return descriptor / np.linalg.norm(descriptor)
    
    @classmethod
    def juntar_descriptores(cls, exclude=None, known_identities=False):
        total = []
        indices_list = []
        
        if known_identities:
            personas = cls.existentes[:6]
        else:
            personas = cls.existentes
        
        for i, persona in enumerate(personas):
            if exclude:
                if i in exclude:
                    continue
            total.extend(persona.descriptores)
            indices_list.extend([i] * len(persona.descriptores))
        
        database = np.array(total)
        indices = np.array(indices_list)
        
        return database, indices
            
    @classmethod
    def encontrar_similar(cls, descriptores, known_identities=False):
        """
        Recibe los descriptores de un nuevo frame.
        
        Si encuentra una persona similar entonces
        retorna el índice de la persona. 
        
        Sino retorna -1
        
        Si se recibe una matriz entonces el output
        es una lista
        """

        # Lista para ser retornada
        results = [-1] * len(descriptores)
        
        # Para mantener registro de la posicion original de descriptores
        original_indeces = list(range(len(descriptores)))
        
        # Mantener registro de que identidades ya fueron usadas
        selected_identities = []
        
        while len(descriptores):
            # Juntar descriptores de personas existentes
            database, indices = cls.juntar_descriptores(exclude=selected_identities, known_identities=known_identities) 
            
            if len(database)==0:
                break

            # Comparar descriptores existentes con descriptores nuevos
            scores = database@descriptores.T
            
            # Encontrar scores maximos
            best_scores = np.max(scores, axis=0)
            ix_best_scores = np.argmax(scores, axis=0) 
            
            # Si ningun score supera el threshold, salir
            if not np.any(best_scores > cls.thr):
                break
            
            # Encontrar descriptor hizo el mejor match en la database
            descriptor_with_best_score = np.argmax(best_scores)
            
            original_ix = original_indeces.pop(descriptor_with_best_score)
            matching_identity = indices[ix_best_scores[descriptor_with_best_score]]
            
            results[original_ix] = matching_identity
            selected_identities.append(matching_identity)
            descriptores = np.delete(descriptores, descriptor_with_best_score, axis=0)
        
        return results
    
    @classmethod
    def nueva_persona(cls, descriptor, frame, index, known_identity=False):
        """
        Agrega una nueva persona a lista de personas
        """
        cls.existentes.append(cls(descriptor, 
                                  frame, 
                                  index, 
                                  known_identity=known_identity))
    
    @classmethod
    def agregar_descriptores(cls, identity, descriptores, frame):
        """
        Revisa los resultados para ver si agregar descriptor a 
        identidad existente o si crea una nueva identidad
        """
        for index_origin, index_target in enumerate(identity):
            descriptor = descriptores[index_origin]
            if index_target >= 0:
                # Agregar descriptor a persona correspondiente
                persona = cls.existentes[index_target]
                persona.add(descriptor, frame, index_origin)
            else:
                # Agregar nueva persona
                cls.nueva_persona(descriptor, frame, index_origin)
                
    @classmethod
    def juntar_personas(cls, known_identities=False):
        """
        Revisa los resultados para ver si agregar descriptor a 
        identidad existente o si crea una nueva identidad
        """
        dataframes = []
        
        NAMES = ['Chandler', 'Joey', 'Monica', 'Phoebe', 'Rachel', 'Ross']
        
        for i, persona in enumerate(cls.existentes):
            data = persona.data.copy()
            if known_identities and i < 6:
                data.loc[:, 'ID'] = NAMES[i]
            else:
                data.loc[:, 'ID'] = str(i)
            dataframes.append(data)
        
        joined = pd.concat(dataframes)
        joined.set_index(['Frame', 'Index'], inplace=True)
        joined.sort_index(inplace=True)
        
        # Rellenar valores de "Mira A (ID)"
        for i, row in joined.iterrows():
            if row['Mira A (Index)'] is not None:
                target = row['Mira A (Index)']
                target_id = joined.loc[(i[0], target), 'ID']
                joined.loc[i, 'Mira A (ID)'] = target_id
        
        return joined
    
    @classmethod
    def reset(cls):
        cls.existentes = []

# Sin conocer identidades

In [170]:
Personas.reset()
for i in tqdm(range(2412)):
    frame = np.load(f"{directory}/{i}/descriptores.npy")
    results = Personas.encontrar_similar(frame)
    Personas.agregar_descriptores(results, frame, i)

In [171]:
len(Personas.existentes)

22

## Algunas personas existentes

In [172]:
len(Personas.existentes[0].descriptores)

1657

In [173]:
Personas.existentes[0].data.head()

,Frame,Index,angry,disgust,scared,happy,sad,surprised,neutral,max_emotion,yaw,pitch,roll,Mira A (Index),bb_x_min,bb_y_min,bb_x_max,bb_y_max
0,0,0,0.0605916,0.00439636,0.163676,0.105976,0.437814,0.00956166,0.217985,sad,52.5783,-22.8564,-9.35359,1,500,57,545,127
1,1,0,0.0358013,0.00336232,0.154518,0.0990854,0.434491,0.00920353,0.263538,sad,52.8774,-24.6822,-11.4884,1,498,55,543,125
2,2,0,0.0296305,0.00571881,0.143816,0.153038,0.269821,0.00970382,0.388271,neutral,56.5237,-31.2142,-16.0661,1,497,54,541,124
3,3,0,0.0447929,0.0214209,0.132077,0.34365,0.189225,0.00770666,0.261127,happy,62.1406,-31.1098,-17.0192,1,496,54,540,123
4,4,0,0.0775742,0.0162405,0.167147,0.129144,0.315516,0.00700236,0.287376,sad,62.1805,-32.8512,-15.408,1,496,54,541,124


In [174]:
Personas.existentes[1].data.head()

,Frame,Index,angry,disgust,scared,happy,sad,surprised,neutral,max_emotion,yaw,pitch,roll,Mira A (Index),bb_x_min,bb_y_min,bb_x_max,bb_y_max
0,0,1,0.129491,0.00111534,0.0463445,0.0273935,0.0849974,0.0105302,0.700128,neutral,-74.909,-2.25067,-3.39456,0,349,100,395,168
1,1,1,0.15072,0.00193867,0.0475138,0.0604405,0.0876748,0.0110678,0.640644,neutral,-66.9448,-6.10984,-3.19043,0,352,101,395,168
2,2,1,0.173361,0.00386459,0.0374429,0.0840561,0.0768111,0.00710518,0.617359,neutral,-67.3275,-7.93867,-3.19107,0,352,101,394,169
3,3,1,0.174754,0.00435174,0.0571104,0.0997265,0.0800052,0.0077135,0.576339,neutral,-67.2842,-5.44004,-9.06762,0,352,102,394,168
4,4,1,0.179711,0.00391713,0.0535906,0.0845384,0.0919304,0.00692739,0.579385,neutral,-66.1598,-7.27341,-9.32238,0,353,101,394,169


In [175]:
Personas.existentes[2].data.head()

,Frame,Index,angry,disgust,scared,happy,sad,surprised,neutral,max_emotion,yaw,pitch,roll,Mira A (Index),bb_x_min,bb_y_min,bb_x_max,bb_y_max
36,36,0,0.187983,0.0191112,0.230469,0.0933016,0.0716735,0.00687361,0.390588,neutral,-64.7788,-3.32716,-0.00700378,2,327,151,371,214
37,37,0,0.172312,0.0151837,0.217673,0.18264,0.0577653,0.0232873,0.331138,neutral,-65.5009,-7.9733,0.671326,2,327,155,370,215
38,38,0,0.164391,0.0139124,0.214681,0.135206,0.0596528,0.0119053,0.400252,neutral,-68.345,-6.59921,1.57845,1,328,154,371,215
39,39,0,0.159896,0.0164637,0.245906,0.0892377,0.064404,0.0104809,0.413612,neutral,-67.822,-4.48047,-1.37256,1,328,154,372,213
40,40,1,0.137869,0.0111043,0.195144,0.127387,0.0553357,0.0106214,0.462538,neutral,-70.8564,-6.59427,3.09396,0,329,155,371,213


## Juntar personas en un dataframe

In [176]:
todos_df = Personas.juntar_personas()
todos_df.head(30)

angry     disgust     scared      happy        sad  \
Frame Index                                                           
0     0      0.0605916  0.00439636   0.163676   0.105976   0.437814   
      1       0.129491  0.00111534  0.0463445  0.0273935  0.0849974   
1     0      0.0358013  0.00336232   0.154518  0.0990854   0.434491   
      1        0.15072  0.00193867  0.0475138  0.0604405  0.0876748   
2     0      0.0296305  0.00571881   0.143816   0.153038   0.269821   
      1       0.173361  0.00386459  0.0374429  0.0840561  0.0768111   
3     0      0.0447929   0.0214209   0.132077    0.34365   0.189225   
      1       0.174754  0.00435174  0.0571104  0.0997265  0.0800052   
4     0      0.0775742   0.0162405   0.167147   0.129144   0.315516   
      1       0.179711  0.00391713  0.0535906  0.0845384  0.0919304   
5     0      0.0871559   0.0245417   0.188441   0.128384    0.32331   
      1       0.188261  0.00328831  0.0645917  0.0810587   0.107985   
6     0      0.0592054     0.02382   0.181719   0.186465   0.297571   
      1       0.194449  0.00565914  0.0682373   0.105163   0.097182   
7     0      0.0826657   0.0225371   0.146828   0.100438   0.298938   
      1       0.165114  0.00310895  0.0551697    0.14672  0.0916012   
8     0      0.0886292  0.00262837   0.154497  0.0424788   0.410849   
      1      0.0925082  0.00777524   0.037213    0.24055  0.0777208   
9     0      0.0967668  0.00271104     0.1374  0.0456216   0.370011   
      1       0.112147  0.00217608  0.0599942  0.0836527   0.154739   
10    0       0.125749  0.00728734   0.160813  0.0909619   0.323544   
      1       0.116017   0.0056258  0.0771178   0.142696   0.180123   
11    0      0.0945341  0.00795435   0.168011  0.0488995   0.394946   
      1       0.139278  0.00345489   0.105192  0.0874053   0.186378   
12    0      0.0815447  0.00748158    0.17063  0.0454334   0.337687   
      1       0.134497   0.0232779  0.0706168    0.31363   0.113147   
13    0      0.0958046  0.00658535   0.153419   0.134145   0.305697   
      1       0.138981  0.00598239    0.10063   0.304767   0.112893   
14    0        0.06242  0.00495147   0.155359   0.116199   0.280255   
      1       0.124511  0.00624969  0.0886631   0.311641   0.118132   

              surprised   neutral max_emotion      yaw    pitch      roll  \
Frame Index                                                                 
0     0      0.00956166  0.217985         sad  52.5783 -22.8564  -9.35359   
      1       0.0105302  0.700128     neutral  -74.909 -2.25067  -3.39456   
1     0      0.00920353  0.263538         sad  52.8774 -24.6822  -11.4884   
      1       0.0110678  0.640644     neutral -66.9448 -6.10984  -3.19043   
2     0      0.00970382  0.388271     neutral  56.5237 -31.2142  -16.0661   
      1      0.00710518  0.617359     neutral -67.3275 -7.93867  -3.19107   
3     0      0.00770666  0.261127       happy  62.1406 -31.1098  -17.0192   
      1       0.0077135  0.576339     neutral -67.2842 -5.44004  -9.06762   
4     0      0.00700236  0.287376         sad  62.1805 -32.8512   -15.408   
      1      0.00692739  0.579385     neutral -66.1598 -7.27341  -9.32238   
5     0      0.00712206  0.241046         sad  60.6969 -24.8229  -8.63445   
      1      0.00837265  0.546443     neutral -64.1349 -11.0594  -8.99152   
6     0      0.00498295  0.246237         sad  58.3753 -20.5374  -5.50766   
      1       0.0137876  0.515523     neutral -68.8196 -12.7188  -4.28578   
7     0      0.00729222  0.341301     neutral  62.3878 -19.5016  -6.98637   
      1       0.0145362   0.52375     neutral -68.7032 -14.0652  -2.16037   
8     0      0.00698513  0.293933         sad  59.7943 -17.0409  -6.45181   
      1      0.00836867  0.535864     neutral -72.3691  -13.728  -1.12181   
9     0      0.00665116  0.340838         sad  58.5405 -16.1177  -6.37704   
      1       0.0154758  0.571815     neutral  -69.655 -13.1049   -2.2541   
10    0      0.00984263  0.281802         sad  63.

In [177]:
todos_df.to_csv('datos_personas.csv')

# Procesar personas conociendo identidades

In [165]:
chandler = np.load('../Identidades Conocidas/Chandler.npy')
joey = np.load('../Identidades Conocidas/Joey.npy')
monica = np.load('../Identidades Conocidas/Monica.npy')
phoebe = np.load('../Identidades Conocidas/Phoebe.npy')
rachel = np.load('../Identidades Conocidas/Rachel.npy')
ross = np.load('../Identidades Conocidas/Ross.npy')

In [166]:
Personas.reset()

Personas.nueva_persona(chandler, None, None, known_identity=True)
Personas.nueva_persona(joey, None, None, known_identity=True)
Personas.nueva_persona(monica, None, None, known_identity=True)
Personas.nueva_persona(phoebe, None, None, known_identity=True)
Personas.nueva_persona(rachel, None, None, known_identity=True)
Personas.nueva_persona(ross, None, None, known_identity=True)


for i in tqdm(range(2412)):
    frame = np.load(f"{directory}/{i}/descriptores.npy")
    results = Personas.encontrar_similar(frame, known_identities=True)
    Personas.agregar_descriptores(results, frame, i)

## Ejemplo: Chandler

In [167]:
Personas.existentes[0].data.head()

,Frame,Index,angry,disgust,scared,happy,sad,surprised,neutral,max_emotion,yaw,pitch,roll,Mira A (Index),bb_x_min,bb_y_min,bb_x_max,bb_y_max
36,36,0,0.187983,0.0191112,0.230469,0.0933016,0.0716735,0.00687361,0.390588,neutral,-64.7788,-3.32716,-0.00700378,2,327,151,371,214
37,37,0,0.172312,0.0151837,0.217673,0.18264,0.0577653,0.0232873,0.331138,neutral,-65.5009,-7.9733,0.671326,2,327,155,370,215
38,38,0,0.164391,0.0139124,0.214681,0.135206,0.0596528,0.0119053,0.400252,neutral,-68.345,-6.59921,1.57845,1,328,154,371,215
39,39,0,0.159896,0.0164637,0.245906,0.0892377,0.064404,0.0104809,0.413612,neutral,-67.822,-4.48047,-1.37256,1,328,154,372,213
40,40,1,0.137869,0.0111043,0.195144,0.127387,0.0553357,0.0106214,0.462538,neutral,-70.8564,-6.59427,3.09396,0,329,155,371,213


## Juntar personas en un dataframe

In [168]:
todos_df = Personas.juntar_personas(known_identities=True)
todos_df.head(30)

angry     disgust     scared      happy        sad  \
Frame Index                                                           
0     0      0.0605916  0.00439636   0.163676   0.105976   0.437814   
      1       0.129491  0.00111534  0.0463445  0.0273935  0.0849974   
1     0      0.0358013  0.00336232   0.154518  0.0990854   0.434491   
      1        0.15072  0.00193867  0.0475138  0.0604405  0.0876748   
2     0      0.0296305  0.00571881   0.143816   0.153038   0.269821   
      1       0.173361  0.00386459  0.0374429  0.0840561  0.0768111   
3     0      0.0447929   0.0214209   0.132077    0.34365   0.189225   
      1       0.174754  0.00435174  0.0571104  0.0997265  0.0800052   
4     0      0.0775742   0.0162405   0.167147   0.129144   0.315516   
      1       0.179711  0.00391713  0.0535906  0.0845384  0.0919304   
5     0      0.0871559   0.0245417   0.188441   0.128384    0.32331   
      1       0.188261  0.00328831  0.0645917  0.0810587   0.107985   
6     0      0.0592054     0.02382   0.181719   0.186465   0.297571   
      1       0.194449  0.00565914  0.0682373   0.105163   0.097182   
7     0      0.0826657   0.0225371   0.146828   0.100438   0.298938   
      1       0.165114  0.00310895  0.0551697    0.14672  0.0916012   
8     0      0.0886292  0.00262837   0.154497  0.0424788   0.410849   
      1      0.0925082  0.00777524   0.037213    0.24055  0.0777208   
9     0      0.0967668  0.00271104     0.1374  0.0456216   0.370011   
      1       0.112147  0.00217608  0.0599942  0.0836527   0.154739   
10    0       0.125749  0.00728734   0.160813  0.0909619   0.323544   
      1       0.116017   0.0056258  0.0771178   0.142696   0.180123   
11    0      0.0945341  0.00795435   0.168011  0.0488995   0.394946   
      1       0.139278  0.00345489   0.105192  0.0874053   0.186378   
12    0      0.0815447  0.00748158    0.17063  0.0454334   0.337687   
      1       0.134497   0.0232779  0.0706168    0.31363   0.113147   
13    0      0.0958046  0.00658535   0.153419   0.134145   0.305697   
      1       0.138981  0.00598239    0.10063   0.304767   0.112893   
14    0        0.06242  0.00495147   0.155359   0.116199   0.280255   
      1       0.124511  0.00624969  0.0886631   0.311641   0.118132   

              surprised   neutral max_emotion      yaw    pitch      roll  \
Frame Index                                                                 
0     0      0.00956166  0.217985         sad  52.5783 -22.8564  -9.35359   
      1       0.0105302  0.700128     neutral  -74.909 -2.25067  -3.39456   
1     0      0.00920353  0.263538         sad  52.8774 -24.6822  -11.4884   
      1       0.0110678  0.640644     neutral -66.9448 -6.10984  -3.19043   
2     0      0.00970382  0.388271     neutral  56.5237 -31.2142  -16.0661   
      1      0.00710518  0.617359     neutral -67.3275 -7.93867  -3.19107   
3     0      0.00770666  0.261127       happy  62.1406 -31.1098  -17.0192   
      1       0.0077135  0.576339     neutral -67.2842 -5.44004  -9.06762   
4     0      0.00700236  0.287376         sad  62.1805 -32.8512   -15.408   
      1      0.00692739  0.579385     neutral -66.1598 -7.27341  -9.32238   
5     0      0.00712206  0.241046         sad  60.6969 -24.8229  -8.63445   
      1      0.00837265  0.546443     neutral -64.1349 -11.0594  -8.99152   
6     0      0.00498295  0.246237         sad  58.3753 -20.5374  -5.50766   
      1       0.0137876  0.515523     neutral -68.8196 -12.7188  -4.28578   
7     0      0.00729222  0.341301     neutral  62.3878 -19.5016  -6.98637   
      1       0.0145362   0.52375     neutral -68.7032 -14.0652  -2.16037   
8     0      0.00698513  0.293933         sad  59.7943 -17.0409  -6.45181   
      1      0.00836867  0.535864     neutral -72.3691  -13.728  -1.12181   
9     0      0.00665116  0.340838         sad  58.5405 -16.1177  -6.37704   
      1       0.0154758  0.571815     neutral  -69.655 -13.1049   -2.2541   
10    0      0.00984263  0.281802         sad  63.

In [169]:
todos_df.to_csv('datos_personas_conocidas.csv')

In [183]:
todos_df

angry      disgust     scared      happy        sad  \
Frame Index                                                            
0     0      0.0605916   0.00439636   0.163676   0.105976   0.437814   
      1       0.129491   0.00111534  0.0463445  0.0273935  0.0849974   
1     0      0.0358013   0.00336232   0.154518  0.0990854   0.434491   
      1        0.15072   0.00193867  0.0475138  0.0604405  0.0876748   
2     0      0.0296305   0.00571881   0.143816   0.153038   0.269821   
      1       0.173361   0.00386459  0.0374429  0.0840561  0.0768111   
3     0      0.0447929    0.0214209   0.132077    0.34365   0.189225   
      1       0.174754   0.00435174  0.0571104  0.0997265  0.0800052   
4     0      0.0775742    0.0162405   0.167147   0.129144   0.315516   
      1       0.179711   0.00391713  0.0535906  0.0845384  0.0919304   
5     0      0.0871559    0.0245417   0.188441   0.128384    0.32331   
      1       0.188261   0.00328831  0.0645917  0.0810587   0.107985   
6     0      0.0592054      0.02382   0.181719   0.186465   0.297571   
      1       0.194449   0.00565914  0.0682373   0.105163   0.097182   
7     0      0.0826657    0.0225371   0.146828   0.100438   0.298938   
      1       0.165114   0.00310895  0.0551697    0.14672  0.0916012   
8     0      0.0886292   0.00262837   0.154497  0.0424788   0.410849   
      1      0.0925082   0.00777524   0.037213    0.24055  0.0777208   
9     0      0.0967668   0.00271104     0.1374  0.0456216   0.370011   
      1       0.112147   0.00217608  0.0599942  0.0836527   0.154739   
10    0       0.125749   0.00728734   0.160813  0.0909619   0.323544   
      1       0.116017    0.0056258  0.0771178   0.142696   0.180123   
11    0      0.0945341   0.00795435   0.168011  0.0488995   0.394946   
      1       0.139278   0.00345489   0.105192  0.0874053   0.186378   
12    0      0.0815447   0.00748158    0.17063  0.0454334   0.337687   
      1       0.134497    0.0232779  0.0706168    0.31363   0.113147   
13    0      0.0958046   0.00658535   0.153419   0.134145   0.305697   
      1       0.138981   0.00598239    0.10063   0.304767   0.112893   
14    0        0.06242   0.00495147   0.155359   0.116199   0.280255   
      1       0.124511   0.00624969  0.0886631   0.311641   0.118132   
...                ...          ...        ...        ...        ...   
2397  1         0.0562   0.00212741   0.113653   0.227121   0.141359   
      2       0.042164  0.000559839   0.329804  0.0509418   0.245738   
2398  0       0.120013  0.000677944  0.0957136   0.296092  0.0734494   
      1      0.0760071   0.00255851   0.163593   0.328888  0.0897436   
2399  0       0.023329  0.000928706  0.0804559   0.359931  0.0556222   
      1       0.133871  0.000614166   0.123738    0.24653  0.0627527   
2400  0      0.0274979   0.00408229   0.045464   0.627025  0.0247763   
      1      0.0553346  4.92919e-05  0.0726192   0.587931  0.0423111   
2401  0      0.0181721   0.00353397  0.0495201   0.652049  0.0322006   
      1      0.0645314  0.000183476   0.143884   0.501296  0.0929761   
2402  0      0.0295337   0.00346504  0.0880064   0.515301  0.0772593   
      1      0.0312751  0.000338677   0.195482  0.0651674   0.113472   
2403  0      0.0615162      0.01642   0.154365   0.438727  0.0808228   
      1      0.0318166  0.000134004  0.0906727   0.214956   0.121546   
2404  0      0.0451101   0.00795801   0.109503   0.493318  0.0888846   
      1      0.0702095  0.000315885   0.138571   0.150494   0.140405   
2405  0        0.20993  0.000483459   0.122563   0.185435   0.181121   
      1      0.0384476   0.00763216  0.0782884   0.564081  0.0696174   
2406  0      0.0360237    0.0106579  0.0772646   0.544259    0.06756   
      1       0.213892  0.000312102   0.122885   0.245511   0.179608   
2407  0      0.0344126   0.00690455  0.0714606   0.531863  0.0845229   
      1       0.371626  0.000417292   0.111102  0.0828517   0.173332   
2408  0      0.0188436   0.00494664  0.0532806   0.648016